I have a csv file with all the date/times of ASTER observations I'm using, and I want to now download all the MODIS (MOD021KM) imagery for the same date/times covering the same spot.

I'm using the Earthdata API because doing this through their web portal would be a slow and cumbersome process.

Some references:
 * [Earthdata API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html)
 * [MOD021KM product](https://ladsweb.modaps.eosdis.nasa.gov/missions-and-measurements/products/MOD021KM/)

In [ ]:
import requests
import json
import pandas as pd
import io
import glob

Look at the ASTER images that I have saved, make a list of them and their timestamps:

In [ ]:
# Specify the directory all my ASTER images are in
aster_directory = "/storage/spestana/ASTER/AST_L1T/geotiff/T/T_band14_Tuolumne-and-CUES/"

# Find all our ASTER files and their timestamps in our directory
aster = aster_utils.aster_timestamps(aster_directory, ext='tif')

# Add a UTC-8 datetime for pacific standard that CUES uses
aster['datetime'] = aster.timestampUTC - pd.Timedelta(hours=8)

# Save a csv file of the ASTER files we are using
#aster.to_csv('aster_AST_L1T_band14_Tuolumne-and-CUES.csv')

# Alternatively, if I already have a csv file somewhere with the ASTER timestamps I want, read it in
#aster = pd.read_csv('aster_AST_L1T_band14_Tuolumne-and-CUES.csv')

Set up some of the granule search criteria for Earthdata search API

In [ ]:
url = "https://cmr.earthdata.nasa.gov/search/granules.csv?" #"https://cmr.earthdata.nasa.gov/search/collections?has_granules_created_at\[\]=2015-01-01T19:00:00Z,"

# Product information for the MODIS products I want
shortname_id = "short_name=MOD021KM" # "short_name=MOD03"
version = "version=6.1"

# bounding box around the study area I want to look at (upper Tuolumne River basin)
# lower left longitude, lower left latitude, upper right longitude, upper right latitude
bounding_box = "bounding_box[]=-120,37,-118,38" 

In [ ]:
# empty list to hold modis download URLs
modis_urls = []

for i, this_timestamp in enumerate(aster.timestampUTC):
       
    # the date is the first 10 characters of the timestamp string
    date = this_timestamp[:10]
    #print(date)
    
    
    # Terra ASTER/MODIS is always flying over here at ~18:50 UTC (day) or ~5:50 UTC (night)
    hours = [(18, 19), (5, 6)]
    for h in hours:
        start_time = "{date}T{hour_start}:00:00Z".format(date=date,hour_start=h[0])
        end_time = "{date}T{hour_end}:00:00Z".format(date=date,hour_end=h[1])
        time_range = "temporal=" + start_time + "," + end_time
    
        # build the whole request URL and make the request.get
        response = requests.get(url+"&"+shortname_id+"&"+version+"&"+bounding_box+"&"+time_range)
    
        # read the response CSV and put in a temporary dataframe
        df = pd.read_csv(io.StringIO(response.text))
        
        for modis_url in df['Online Access URLs']:
            modis_urls.append(modis_url)

Write the list of URLs out to a text file:

In [ ]:
with open("modis_download_list.txt", "w") as output:
    for row in modis_urls:
        output.write(str(row) + '\n')

---
**Download using wget**

`wget --http-user=YOUR_USERNAME --ask-password --keep-session-cookies --auth-no-challenge=on -c -i modis_download_list.txt`

---

#### *WORK IN PROGRESS*

**Convert HDF to GeoTiff**

First, get filepaths for the MODIS observations I've just now downloaded:

In [ ]:
# search for MODIS files in this directory
modis_dir = '/storage/MODIS/Tuolumne_MOD021KM/'
# get .hdf files here
modis_filelist = glob.glob(modis_dir + '*.hdf', recursive=True)

In [ ]:
import rioxarray
from pyproj import Transformer
from affine import Affine
from rioxarray.rioxarray import affine_to_coords

def modis_hdf2geotiff(modis_filename, modis_variable, output_filename):
    
    '''Convert a MODIS HDF file to GeoTiff and reproject following this method: https://gis.stackexchange.com/questions/345691/using-python-gdal-to-reproject-an-hdf'''
    
    modis_crs_str = "+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    
    rds = rioxarray.open_rasterio(modis_filename,
                                  variable=[modis_variable],
                                  parse_coordinates=False)
    
    transformer = Transformer.from_crs("EPSG:4326", modis_crs_str, always_xy=True)
    west, north = transformer.transform(rds.WESTBOUNDINGCOORDINATE, rds.NORTHBOUNDINGCOORDINATE)
    
    pixel_size = 1000 # 1 km
    transform = Affine(pixel_size, 0, west, 0, -pixel_size, north)
    
    coords = affine_to_coords(transform, rds.rio.width, rds.rio.height)
    rds.coords["x"] = coords["x"]
    rds.coords["y"] = coords["y"]
    
    # add the CRS
    rds.rio.write_crs(modis_crs_str, inplace=True)
    
    rds4326 = rds.rio.reproject("EPSG:4326")
    
    # write out to geotiff file
    rds4326[modis_variable].rio.to_raster(output_filename)
    
    return None

In [ ]:
modis_variable = "4"

for i in range(len(modis_filelist)):
    fn = "{}".format(modis_filelist[i])
    new_fn = "{}.tif".format(modis_filelist[i][:-4])
    print('\nConverting {} to {}\n'.format(fn, new_fn))
    modis_hdf2geotiff(fn, modis_variable, new_fn)